In [102]:
import numpy as np
from IPython.display import clear_output
import random

class PossibleMove:
    
    def __init__(self,cell,numbers):
        self.cell=cell
        self.numbers=np.copy(numbers)
        np.random.shuffle(self.numbers)
        self.index=0
        
    def __iter__(self):
        self.index=0
        return self
    
    def __next__(self):
        if len(self.numbers)>self.index:
            self.index+=1
            return self.cell,self.numbers[self.index-1]
        else:
            raise StopIteration
            
    def __lt__(self, other):
        if len(self.numbers) == len(other.numbers):
            return random.choice([-1,1])
        return len(self.numbers) < len(other.numbers)

class PossibleMoves:
    
    def __init__(self,sudoku):
        self.moves=np.copy(sudoku.moves)
        self.board=np.copy(sudoku.board)
        availables=[]
        for y in range(9):
            for x in range(9):
                if self.board[y,x]==0:
                    possibilities=[]
                    possibilities=self.moves[y,x,:]*[1,2,3,4,5,6,7,8,9]
                    possibilities=possibilities[possibilities>0]
                    if len(possibilities)>1:
                        availables.append(PossibleMove((y,x),possibilities))
        availables.sort()
        self.availables=availables
        self.index=0
    
    def __iter__(self):
        self.index=0
        self.current=None
        return self

    def __next__(self):
        while len(self.availables)>self.index:
            if self.current==None:
                self.current=iter(self.availables[self.index])
            try:
                this=next(self.current)
            except StopIteration:
                self.current=None
            
            if not self.current==None:
                return this
            else:
                self.index+=1
        
        raise StopIteration
                    
        
class Sudoku:
    
    def __init__(self,board):
        self.size=9
        self.board=board#np.zeros((self.size,self.size))

    def checkSquare(self,row,col,n):
        col*=3
        row*=3
        for yy in range(3):
            for xx in range(3):
                if self.board[row+yy,col+xx]==n:
                    return True
        return False
    
    def checkColumn(self,col,n):
        for yy in range(self.size):
            if self.board[yy,col]==n:
                return True
        return False
    
    def checkRow(self,row,n):
        for xx in range(self.size):
            if self.board[row,xx]==n:
                return True
        return False
        
    def findPossibleSolutions(self):
        moves=np.zeros((self.size,self.size,self.size))
        for y in range(self.size):
            for x in range(self.size):
                if self.board[y,x]==0:
                    for n in range(self.size):
                        exists=self.checkSquare(int(y/3),int(x/3),n+1)
                        if not exists:
                            exists=self.checkColumn(x,n+1)
                        if not exists:
                            exists=self.checkRow(y,n+1)
                        if not exists:
                            moves[y,x,n]=1
        self.moves=moves
    
    def simplify(self):
        pass
    
    def checkEnd(self):
        for y in range(self.size):
            for x in range(self.size):
                if self.board[y,x]==0:
                    return False
        return True
    
    def checkValid(self):
        for y in range(self.size):
            for x in range(self.size):
                row=int(y/3)
                col=int(x/3)
                n=self.board[y,x]
                if n>0:
                    self.board[y,x]=0
                    exists=self.checkSquare(row,col,n)
                    if not exists:
                        exists=self.checkColumn(x,n)
                    if not exists:
                        exists=self.checkRow(y,n)
                    self.board[y,x]=n
                    if exists:
                        return False
        return True
    
    def tryFill(self):
        find=False
        #cerca soluzioni uniche
        for y in range(self.size):
            for x in range(self.size):
                if self.board[y,x]==0:
                    nn=0
                    moves=0
                    for n in range(self.size):
                        if self.moves[y,x,n]==1:
                            nn=n+1
                            moves+=1
                    if moves==1:
                        self.board[y,x]=nn
                        find=True
        #cerca unica soluzione possibile per quadrato
        for y in range(int(self.size/3)):
            for x in range(int(self.size/3)):
                for n in range(self.size):
                    exists=self.checkSquare(y,x,n+1)
                    if not exists:
                        possibility=0
                        cell=(-1,-1)
                        for yy in range(3):
                            for xx in range(3):
                                if self.board[y*3+yy,x*3+xx]==0 and self.moves[y*3+yy,x*3+xx,n]==1:
                                    cell=(y*3+yy,x*3+xx)
                                    possibility+=1                                    
                        if possibility==1:
                            self.board[cell[0],cell[1]]=n+1
                            find=True    

        #cerca unica soluzione possibile per riga
        for y in range(self.size):
            for n in range(self.size):
                exists=self.checkRow(y,n+1)
                if not exists:
                    possibility=0
                    cell=(-1,-1)
                    for x in range(self.size):
                        if self.board[y,x]==0 and self.moves[y,x,n]==1:
                            cell=(y,x)
                            possibility+=1
                    if possibility==1:
                        self.board[cell[0],cell[1]]=n+1
                        find=True    

        #cerca unica soluzione possibile per colonna
        for x in range(self.size):
            for n in range(self.size):
                exists=self.checkColumn(x,n+1)
                if not exists:
                    possibility=0
                    cell=(-1,-1)
                    for y in range(self.size):
                        if self.board[y,x]==0 and self.moves[y,x,n]==1:
                            cell=(y,x)
                            possibility+=1
                    if possibility==1:
                        self.board[cell[0],cell[1]]=n+1
                        find=True    
        
                        
        return find
    
    def printBoard(self):
        output="|-------|-------|-------|\n"
        for y in range(self.size):            
            for x in range(self.size):
                if (x%3)==0:
                    output+="| "
                if self.board[y,x]>0:
                    output+=str(self.board[y,x])+" "
                else:
                    output+=". "
            output+="|\n"
            if y>1 and((y+1)%3==0 or y==8):
                output+="|-------|-------|-------|\n"

        print(output)
        print("\n")

    def printMoves(self):
        output=""
        for y in range(self.size):
            for x in range(self.size):
                for n in range(self.size):
                    output+=str(int(self.moves[y,x,n]))
                output+=" "
            output+='\n'
        print(output)
        print("---------\n\n")
    
    def solve(self,level=0,start=None):
        
        ended=False
        filled=False
        valided=False
        steps=0
        doneLevel=level
        if level==0:
            start = timeit.default_timer()
        
        while not ended:
            #clear_output(wait=True)
            steps+=1
            print("level:{} steps:{} time:{:.4f}".format(level,steps,timeit.default_timer() - start))
            self.findPossibleSolutions()
            #self.printMoves()
            filled=self.tryFill()
            #self.printBoard()
            #time.sleep(2)
            ended=self.checkEnd()
            valided=self.checkValid()
            if valided:
                if (not ended) and (not filled):
                    child=PossibleMoves(self)

                    for move,number in child:
                        board=Sudoku(child.board)
                        board.board[move[0],move[1]]=number
                        done=False
                        if board.checkValid():
                            done,doneLevel=board.solve(level+1,start)
                        if done:
                            if level>0:
                                return True,doneLevel
                            else:
                                valided=True
                                ended=True
                                break
                    break
                
        if doneLevel==level:
            #print(level,steps)
            self.printBoard()

        if level==0:
            #s.printMoves()
            stop = timeit.default_timer()
            print('Time: ', stop - start)
            if ended and valided:
                print("resolved")
            else:
                print("no solution at the moment")
        
        return ended and valided,level


        


#facile
board=np.array([[9,1,7,2,5,4,0,0,0],
                [4,0,2,0,8,0,0,0,0],
                [6,5,0,0,0,3,4,0,0],
                [0,0,3,0,9,0,2,5,6],
                [5,0,0,7,0,0,3,0,9],
                [2,0,0,0,0,5,0,7,1],
                [0,2,0,5,3,0,7,6,0],
                [3,7,0,1,6,0,0,9,8],
                [0,0,0,0,0,0,0,3,0]])

#difficile
board=np.array([[0,0,0,0,5,1,0,0,0],
                [5,6,1,9,0,0,0,0,0],
                [4,0,0,7,0,0,0,0,0],
                [0,0,2,0,0,5,4,0,0],
                [0,4,5,0,0,0,0,0,8],
                [1,9,0,0,4,0,0,0,3],
                [0,8,0,0,2,7,0,3,1],
                [6,0,0,0,0,0,0,2,0],
                [0,5,0,8,0,0,6,4,9]])

#difficile
board=np.array([[0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0]])


import time 
import timeit


s=Sudoku(board)
s.printBoard()
s.solve()


        

        

|-------|-------|-------|
| . . . | . . . | . . . |
| . . . | . . . | . . . |
| . . . | . . . | . . . |
|-------|-------|-------|
| . . . | . . . | . . . |
| . . . | . . . | . . . |
| . . . | . . . | . . . |
|-------|-------|-------|
| . . . | . . . | . . . |
| . . . | . . . | . . . |
| . . . | . . . | . . . |
|-------|-------|-------|



level:0 steps:1 time:0.0000
level:1 steps:1 time:0.0117
level:2 steps:1 time:0.0240
level:3 steps:1 time:0.0344
level:4 steps:1 time:0.0446
level:5 steps:1 time:0.0553
level:6 steps:1 time:0.0653
level:7 steps:1 time:0.0750
level:8 steps:1 time:0.0845
level:8 steps:2 time:0.0928
level:9 steps:1 time:0.1018
level:10 steps:1 time:0.1110
level:11 steps:1 time:0.1199
level:12 steps:1 time:0.1287
level:13 steps:1 time:0.1373
level:13 steps:2 time:0.1448
level:14 steps:1 time:0.1533
level:15 steps:1 time:0.1616
level:15 steps:2 time:0.1689
level:16 steps:1 time:0.1770
level:17 steps:1 time:0.1851
level:17 steps:2 time:0.1921
level:18 steps:1 time:0.2001
lev

(True, 0)

In [61]:
a=np.array([[[1,0,1,0,0,0,0,0,0]]])
b=a[0,0,:]*[1,2,3,4,5,6,7,8,9]
b=b[b>0]

In [62]:
b

array([1, 3])